### Creating the adjacency list from adjacency matrix

#### open dataframe

In [1129]:
import pandas as pd
import copy
import math
import random

In [1130]:
# import dataframe
data = pd.read_csv('attack_defense_matrix.csv')
data.head()

,player_name,a.j. price,aaron brooks,aaron gordon,adreian payne,al horford,al jefferson,alan anderson,alec burks,alex kirk,...,wesley matthews,will barton,will cherry,willie green,wilson chandler,xavier henry,zach lavine,zach randolph,zaza pachulia,zoran dragic
0,a.j. price,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,aaron brooks,4,0,0,0,4,4,2,0,0,...,0,0,0,3,5,0,3,0,0,0
2,aaron gordon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
3,adreian payne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,al horford,0,0,0,0,0,21,2,0,0,...,2,0,0,0,0,0,2,4,12,0


In [1131]:
data_players = pd.read_csv('players_old.csv')
data_kmeans = pd.read_csv('players_kmeans.csv')
data_players.head()

,player_name,total-close-made,total-mid-made,total-long-made,total-threes-made,total-made,avg-shot-distance
0,al jefferson,212,122,27,19,380,9.275788
1,gerald henderson,79,54,83,26,242,14.483544
2,kemba walker,94,41,63,68,266,14.524186
3,gordon hayward,139,66,80,93,378,14.648619
4,enes kanter,234,28,39,15,316,8.534851


In [1132]:
data_dbpm = pd.read_csv('adv_shot_logs.csv')
data_dbpm.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id,player_height,TS%,DBPM
0,21400010,"OCT 29, 2014 - CHI @ NYK",A,W,24,3,2,10:23,18.1,5,...,"Acy, Quincy",203112,2.3,1,2,aaron brooks,201166,183.0,0.534,-2.4
1,21400447,"DEC 27, 2014 - CHI vs. NOP",H,W,7,2,1,0:20,3.4,7,...,"Ajinca, Alexis",201582,7.1,0,0,aaron brooks,201166,183.0,0.534,-2.4
2,21400447,"DEC 27, 2014 - CHI vs. NOP",H,W,7,4,2,9:43,14.0,3,...,"Ajinca, Alexis",201582,4.2,0,0,aaron brooks,201166,183.0,0.534,-2.4
3,21400447,"DEC 27, 2014 - CHI vs. NOP",H,W,7,13,4,11:45,9.0,5,...,"Ajinca, Alexis",201582,2.2,1,2,aaron brooks,201166,183.0,0.534,-2.4
4,21400379,"DEC 18, 2014 - CHI vs. NYK",H,W,6,4,2,7:21,15.0,10,...,"Aldrich, Cole",202332,5.0,1,2,aaron brooks,201166,183.0,0.534,-2.4


#### assigning IDs and creating nodes

In [1133]:
ids = {}
ids_rev = {}
i = 0
# give each player an ID
for player in data['player_name']:
    ids[player] = (int)(i)
    ids_rev[i] = player
    # construct nodes
    #entry = {}
    #entry['player'] = player
    #entry['id'] = (int)(i)
    i += 1
    
        
    #try:
    #    influence = data_players[player]
    #except KeyError:
    #    influence = 5
    #print('whoops')
    #entry['influence'] = influence # @ total points
    #entry['zone'] = 1 # @ category of average shot distance
    #nodes.append(copy.deepcopy(entry))

In [1134]:
players = list(data.columns.values)
players.pop(0)

'player_name'

#### attack edges

In [1135]:
#data_players['player_name']
def is_in_dataframe(playername):
    al_jef_yes = sum( data_players['player_name'][ii] == playername for ii in range(data_players.shape[0]))
    if al_jef_yes == 1:
        return True
    else:
        return False
def is_in_dataframe_2(playername):
    if playername in data.columns:
        return True
    else:
        return False
    
def find_max_shots(playername):
    if is_in_dataframe(playername):
        return int(data_players.loc[data_players['player_name'] == playername]['total-made'])
    else:
        return 5
def find_max_shots_on(playername):
    if is_in_dataframe_2(playername):
        return data[playername].sum()
    else:
        return 5    
    
def find_type_of_player(playername):
    if is_in_dataframe(playername):
        value1 = int(data_players.loc[data_players['player_name'] == playername]['total-close-made'])
        value2 = int(data_players.loc[data_players['player_name'] == playername]['total-mid-made'])
        value3 = int(data_players.loc[data_players['player_name'] == playername]['total-long-made'])
        value4 = int(data_players.loc[data_players['player_name'] == playername]['total-threes-made'])
        value2 += value3
        
        if value1 >= value2 and value1 >= value3 and value1 >= value4:
            return 1
        else:
            if value2 >= value3 and value2 >= value4:
                return 2
            else:
                if value3 >= value4:
                    return 2
                else:
                    return 3
    else:
        return random.randint(1, 3)
def find_kmeans_of_player(playername):
    if is_in_dataframe(playername):
        return int(data_kmeans.loc[data_kmeans['player_name'] == playername]['new_cat'])
    else:
        return random.randint(1, 3)

In [1136]:
##### construct links - only half of the adjacency matrix
links = []
include_these_players = []
for i in range(0,475):
    for j in range(i+1,475):
        if (int)(data.iloc[i,j]) > 15:
            include_these_players.append(i)
            include_these_players.append(j)
            entry = {}
            #print(type(entry))
            entry['source'] = (int)(i) # player id 1
            entry['target'] = (int)(j) # player id 2
            entry['weight'] = (int)(data.iloc[i,j])*3 + (int)(data.iloc[j,i])*3 # number in the lookup table
            #print(type(entry['source']))
            links.append(copy.deepcopy(entry))

In [1137]:

nodes = []
for id_num in set(include_these_players):
    entry = {}
    entry['player'] = ids_rev[id_num]
    entry['id'] = id_num
    entry['influence'] = find_max_shots(ids_rev[id_num])/20 + 10 # @ total points
    entry['zone'] = find_kmeans_of_player(ids_rev[id_num]) # @ category of average shot distance
    dbpm = data_dbpm.loc[data_dbpm['player_name'] == ids_rev[id_num]]['DBPM']
    #print(ids_rev[id_num])
    #print(dbpm)
    if dbpm.empty:
        entry['dbpm'] = 0
    else:
        entry['dbpm'] = data_dbpm.loc[data_dbpm['player_name'] == ids_rev[id_num]]['DBPM'].iloc[0] #find DBPM from somewhere
    nodes.append(copy.deepcopy(entry))

#print(include_these_players)
#print(links)

for link in links:
    if link['source'] in include_these_players and link['target'] in include_these_players:
        print(str(link['source']) + " " + str(link['target']) + " " + str(link['weight']))



In [1138]:
adj_list['nodes'] = nodes
adj_list['links'] = links
#adj_list

In [1139]:
#once we have our adjacency list, we want to remap the player IDs to a sequential list starting at 1

player_map = {} 
ii = 0
for entry in adj_list['nodes']:
    this_id = entry['id']
    player_map[this_id] = ii
    entry['id'] = player_map[this_id]
    ii = ii+1;
for entry in adj_list['links']:
    entry['source'] = player_map[entry['source']]
    entry['target'] = player_map[entry['target']]
    

In [1140]:
adj_list

{'links': [{'source': 0, 'target': 2, 'weight': 63},
  {'source': 0, 'target': 8, 'weight': 60},
  {'source': 0, 'target': 41, 'weight': 48},
  {'source': 0, 'target': 90, 'weight': 87},
  {'source': 0, 'target': 136, 'weight': 72},
  {'source': 0, 'target': 142, 'weight': 48},
  {'source': 0, 'target': 186, 'weight': 54},
  {'source': 1, 'target': 9, 'weight': 48},
  {'source': 1, 'target': 28, 'weight': 63},
  {'source': 1, 'target': 91, 'weight': 66},
  {'source': 1, 'target': 112, 'weight': 54},
  {'source': 1, 'target': 132, 'weight': 60},
  {'source': 1, 'target': 139, 'weight': 99},
  {'source': 1, 'target': 151, 'weight': 72},
  {'source': 1, 'target': 155, 'weight': 84},
  {'source': 1, 'target': 163, 'weight': 48},
  {'source': 1, 'target': 177, 'weight': 60},
  {'source': 1, 'target': 186, 'weight': 72},
  {'source': 3, 'target': 6, 'weight': 48},
  {'source': 4, 'target': 22, 'weight': 48},
  {'source': 4, 'target': 140, 'weight': 48},
  {'source': 5, 'target': 26, 'weight'

In [1141]:
import json
with open('json_adj_list_attack.json', 'w') as outfile:
    json.dump(adj_list, outfile)

#### defense edges

In [1142]:
adj_list = {}
links = []

In [1143]:
# construct links - only half of the adjacency matrix
links = []
include_these_players = []
for i in range(0,475):
    for j in range(i+1,475):
        if (int)(data.iloc[i,j]) > 15:
            include_these_players.append(i)
            include_these_players.append(j)
            entry = {}
            #print(type(entry))
            entry['source'] = (int)(i) # player id 1
            entry['target'] = (int)(j) # player id 2
            entry['weight'] = (int)(data.iloc[i,j])*3 + (int)(data.iloc[j,i])*3 # number in the lookup table
            #print(type(entry['source']))
            links.append(copy.deepcopy(entry))

In [1144]:

nodes = []
for id_num in set(include_these_players):
    entry = {}
    entry['player'] = ids_rev[id_num]
    entry['id'] = id_num
    entry['influence'] = find_max_shots_on(ids_rev[id_num])/20  # @ total points
    entry['zone'] = find_kmeans_of_player(ids_rev[id_num]) # @ category of average shot distance
    dbpm = data_dbpm.loc[data_dbpm['player_name'] == ids_rev[id_num]]['DBPM']
    #print(ids_rev[id_num])
    #print(dbpm)
    if dbpm.empty:
        entry['dbpm'] = 0
    else:
        entry['dbpm'] = data_dbpm.loc[data_dbpm['player_name'] == ids_rev[id_num]]['DBPM'].iloc[0] #find DBPM from somewhere
    nodes.append(copy.deepcopy(entry))

In [1145]:
adj_list['nodes'] = nodes
adj_list['links'] = links

In [1146]:
#once we have our adjacency list, we want to remap the player IDs to a sequential list starting at 1

player_map = {} 
ii = 0
for entry in adj_list['nodes']:
    this_id = entry['id']
    player_map[this_id] = ii
    entry['id'] = player_map[this_id]
    ii = ii+1;
for entry in adj_list['links']:
    entry['source'] = player_map[entry['source']]
    entry['target'] = player_map[entry['target']]

In [1147]:
import json
with open('json_adj_list_defense.json', 'w') as outfile:
    json.dump(adj_list, outfile)